<a href="https://colab.research.google.com/github/ntmdrgl/CSCI-164/blob/main/csci_164_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1G5PCNURPLO-mxdOWAoBjfhxDRRwzL79t
!gdown 1B3utxcfUEWVoUTCDxAtyvUCoHYV4iEgy

Downloading...
From: https://drive.google.com/uc?id=1G5PCNURPLO-mxdOWAoBjfhxDRRwzL79t
To: /content/student-mat.csv
100% 57.0k/57.0k [00:00<00:00, 88.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1B3utxcfUEWVoUTCDxAtyvUCoHYV4iEgy
To: /content/letter-recognition.csv
100% 713k/713k [00:00<00:00, 9.15MB/s]


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

import warnings
warnings.filterwarnings(action = 'ignore')

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
student = pd.read_csv('student-mat.csv', sep=';')

student.isnull().sum()
student.dropna()
student.drop_duplicates()

# Numerical columns
num_features = [
    'age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures',
    'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'
]

# Categorical columns
cat_features = [
    'school', 'sex', 'address', 'famsize', 'Pstatus',
    'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup',
    'famsup', 'paid', 'activities', 'nursery', 'higher',
    'internet', 'romantic'
]

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

x_student = student.drop(columns=['G1', 'G2', 'G3'])
y_student = student[['G3']]

x_preprocessed = preprocessor.fit_transform(x_student)

Student Data Preprocessing

In [ ]:
letter = pd.read_csv('letter-recognition.csv')

# data pre-processing
letter.isnull().sum()
letter.dropna()
letter.drop_duplicates()

# feature-transformation

